In [1]:
import cv2
import tensorflow as tf
import numpy as np
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import gen_array_ops
from tensorflow.python.framework import ops
import math

In [2]:
class DeHazeNetCNNModel(object):
    def __init__(self, inputs): 
        self.batch_inputs = tf.reshape(tf.cast(inputs, tf.float32), [-1, 16, 16, 3])
        self.filters_1 = tf.get_variable("DehazeNet/filters_1",
                                        dtype=tf.float32,
                                        initializer=tf.constant(np.load("DehazeNet/numpy_parameters/conv1_0.npy"),
                                                                shape=[5, 5, 3, 20]))
        filter1_pad= tf.zeros([5, 5, 3, 5], dtype=tf.float32)
        filter1_b = tf.concat([self.filters_1, filter1_pad], 3) #[5, 5, 3, 25]
        filter1_c = tf.split(filter1_b, num_or_size_splits=25, axis=3)  # 25 x [5, 5, 3, 1]
        filter1_row0 = tf.concat(filter1_c[0:5], 0)    # [25, 5, 3, 1]
        filter1_row1 = tf.concat(filter1_c[5:10], 0)   # [25, 5, 3, 1]
        filter1_row2 = tf.concat(filter1_c[10:15], 0)  # [25, 5, 3, 1]
        filter1_row3 = tf.concat(filter1_c[15:20], 0)  # [25, 5, 3, 1]
        filter1_row4 = tf.concat(filter1_c[20:25], 0)  # [25, 5, 3, 1]
        filter1_d = tf.concat([filter1_row0, filter1_row1, filter1_row2, filter1_row3, filter1_row4], 1) # [25, 25, 3, 1]
        filter1_final_summary = tf.reshape(filter1_d, [1, 25, 25, 3])
        print(filter1_final_summary)
        self.bias_1 = tf.get_variable("DehazeNet/bias_1",
                                    dtype=tf.float32,
                                    initializer=tf.constant(np.load("DehazeNet/numpy_parameters/conv1_1.npy")))
        self.conv_1 = tf.nn.conv2d(self.batch_inputs, self.filters_1, [1, 1, 1, 1], padding="VALID")
        print(self.conv_1)
        self.output_of_relu_of_sum_of_conv_1_AND_bias_1 = tf.nn.relu(tf.nn.bias_add(self.conv_1, self.bias_1))

        def maxout(inputs, num_units, axis):
            inputs = ops.convert_to_tensor(inputs)
            shape = inputs.get_shape().as_list()
            num_channels = shape[axis]
            shape[axis] = -1
            shape += [num_channels // num_units]
            for i in range(len(shape)):
                if shape[i] is None:
                    shape[i] = gen_array_ops.shape(inputs)[i]
            outputs = math_ops.reduce_max(gen_array_ops.reshape(inputs, shape), -1, keep_dims=False)
            return outputs
        
        self.max_1 = maxout(tf.reshape(self.output_of_relu_of_sum_of_conv_1_AND_bias_1, [1, 12, 12, 20]), 4, 3)

        self.filters_2 = tf.get_variable("DehazeNet/filters_2",
                                    dtype=tf.float32,
                                    initializer=tf.constant(np.load("DehazeNet/numpy_parameters/conv2-1x1_0.npy"), shape=[1, 1, 4, 16]))

        filter2_c = tf.split(self.filters_2, num_or_size_splits=16, axis=3)   # 16 x [1, 1, 4, 1]
        filter2_row0 = tf.concat(filter2_c[0:4], 0)   # [4, 1, 4, 1]
        filter2_row1 = tf.concat(filter2_c[4:8], 0)   # [4, 1, 4, 1]
        filter2_row2 = tf.concat(filter2_c[8:12], 0)  # [4, 1, 4, 1]
        filter2_row3 = tf.concat(filter2_c[12:16], 0) # [4, 1, 4, 1]
        filter2_d = tf.concat([filter2_row0, filter2_row1, filter2_row2, filter2_row3], 1) # [4, 4, 4, 1]
        filter2_final_summary = tf.reshape(filter2_d, [1, 4, 4, 4])
        print(filter2_final_summary)
        self.bias_2 = tf.get_variable("DehazeNet/bias_2",
                                dtype=tf.float32,
                                initializer=tf.constant(np.load("DehazeNet/numpy_parameters/conv2-1x1_1.npy")))
        self.conv_2 = tf.nn.conv2d(tf.reshape(self.max_1, [1, 12, 12, 4]), self.filters_2, [1, 1, 1, 1], padding="VALID")
        self.output_of_relu_of_sum_of_conv_2_AND_bias_2 = tf.nn.relu(tf.nn.bias_add(self.conv_2, self.bias_2))
        
        self.filters_3 = tf.get_variable("DehazeNet/filters_3",
                                        dtype=tf.float32,
                                        initializer=tf.constant(np.load("DehazeNet/numpy_parameters/conv2-3x3_0.npy"), shape=[3, 3, 4, 16]))
        filter3_c = tf.split(self.filters_3, num_or_size_splits=16, axis=3)   # 16 x [1, 1, 4, 1]
        filter3_row0 = tf.concat(filter3_c[0:4], 0)   # [12, 3, 4, 1]
        filter3_row1 = tf.concat(filter3_c[4:8], 0)   # [12, 3, 4, 1]
        filter3_row2 = tf.concat(filter3_c[8:12], 0)  # [12, 3, 4, 1]
        filter3_row3 = tf.concat(filter3_c[12:16], 0) # [12, 3, 4, 1]
        filter3_d = tf.concat([filter3_row0, filter3_row1, filter3_row2, filter3_row3], 1) # [12, 12, 4, 1]
        filter3_final_summary = tf.reshape(filter3_d, [1, 12, 12, 4])
        print(filter3_final_summary)
        self.bias_3 = tf.get_variable("DehazeNet/bias_3",
                                    dtype=tf.float32,
                                    initializer=tf.constant(np.load("DehazeNet/numpy_parameters/conv2-3x3_1.npy")))
        self.conv_3 = tf.nn.conv2d(tf.reshape(self.max_1, [1, 12, 12, 4]), self.filters_3, [1, 1, 1, 1], padding="SAME")
        self.output_of_relu_of_sum_of_conv_3_AND_bias_3 = tf.nn.relu(tf.nn.bias_add(self.conv_3, self.bias_3))
        
        
        self.filters_4 = tf.get_variable("DehazeNet/filters_4",
                                        dtype=tf.float32,
                                        initializer=tf.constant(np.load("DehazeNet/numpy_parameters/conv2-5x5_0.npy"), shape=[5, 5, 4, 16]))
        filter4_c = tf.split(self.filters_4, num_or_size_splits=16, axis=3)   # 16 x [5, 5, 4, 1]
        filter4_row0 = tf.concat(filter4_c[0:4], 0)   # [20, 5, 4, 1]
        filter4_row1 = tf.concat(filter4_c[4:8], 0)   # [20, 5, 4, 1]
        filter4_row2 = tf.concat(filter4_c[8:12], 0)  # [20, 5, 4, 1]
        filter4_row3 = tf.concat(filter4_c[12:16], 0) # [20, 5, 4, 1]
        filter4_d = tf.concat([filter4_row0, filter4_row1, filter4_row2, filter4_row3], 1) # [20, 20, 4, 1]
        filter4_final_summary = tf.reshape(filter4_d, [1, 20, 20, 4])
        print(filter4_final_summary)
        self.bias_4 = tf.get_variable("DehazeNet/bias_4",
                                    dtype=tf.float32,
                                    initializer=tf.constant(np.load("DehazeNet/numpy_parameters/conv2-5x5_1.npy")))
        self.conv_4 = tf.nn.conv2d(tf.reshape(self.max_1, [1, 12, 12, 4]), self.filters_4, [1, 1, 1, 1], padding="SAME")
        self.output_of_relu_of_sum_of_conv_4_AND_bias_4 = tf.nn.relu(tf.nn.bias_add(self.conv_4, self.bias_4)) 


        self.filters_5 = tf.get_variable("DehazeNet/filters_5",
                                        dtype=tf.float32,
                                        initializer=tf.constant(np.load("DehazeNet/numpy_parameters/conv2-7x7_0.npy"), shape=[7, 7, 4, 16]))
        filter5_c = tf.split(self.filters_5, num_or_size_splits=16, axis=3)   # 16 x [7, 7, 4, 1]
        filter5_row0 = tf.concat(filter5_c[0:4], 0)   # [28, 7, 4, 1]
        filter5_row1 = tf.concat(filter5_c[4:8], 0)   # [28, 7, 4, 1]
        filter5_row2 = tf.concat(filter5_c[8:12], 0)  # [28, 7, 4, 1]
        filter5_row3 = tf.concat(filter5_c[12:16], 0) # [28, 7, 4, 1]
        filter5_d = tf.concat([filter5_row0, filter5_row1, filter5_row2, filter5_row3], 1) # [28, 28, 4, 1]
        filter5_final_summary = tf.reshape(filter5_d, [1, 28, 28, 4])
        print(filter5_final_summary)
        self.bias_5 = tf.get_variable("DehazeNet/bias_5",
                                    dtype=tf.float32,
                                    initializer=tf.constant(np.load("DehazeNet/numpy_parameters/conv2-7x7_1.npy")))
        self.conv_5 =  tf.nn.conv2d(tf.reshape(self.max_1, [1, 12, 12, 4]), self.filters_5, [1, 1, 1, 1], padding="SAME")
        print(self.conv_5)
        self.output_of_relu_of_sum_of_conv_5_AND_bias_5 = tf.nn.relu(tf.nn.bias_add(self.conv_5, self.bias_5))
        print(self.output_of_relu_of_sum_of_conv_5_AND_bias_5)

        self.concat_convs_with_16_filters = tf.concat([self.output_of_relu_of_sum_of_conv_2_AND_bias_2,
                                                        self.output_of_relu_of_sum_of_conv_3_AND_bias_3,
                                                        self.output_of_relu_of_sum_of_conv_4_AND_bias_4,
                                                        self.output_of_relu_of_sum_of_conv_5_AND_bias_5], axis=3)
        self.max_pool_1 = tf.nn.max_pool(tf.nn.relu(self.concat_convs_with_16_filters), [1, 8, 8, 1], [1, 1, 1, 1], padding="VALID")

        self.final_conv = tf.layers.conv2d(inputs=self.max_pool_1,
                                    filters=1,
                                    kernel_size=[5, 5],
                                    padding="VALID",
                                    activation=tf.nn.relu,
                                    name="DehazeNet/final_conv")
        self.logits = tf.reshape(self.final_conv, [-1, 1])

In [3]:
class DeHazeNetCNNModelTransmission(object):
    def __init__(self, inputs): 
        self.batch_inputs = tf.reshape(tf.cast(inputs, tf.float32), [-1, 224+15, 224+15, 3])
        self.filters_1 = tf.get_variable("DehazeNet/filters_1",
                                        dtype=tf.float32,
                                        initializer=tf.constant(np.load("DehazeNet/numpy_parameters/conv1_0.npy"),
                                                                shape=[5, 5, 3, 20]))
        
        self.bias_1 = tf.get_variable("DehazeNet/bias_1",
                                    dtype=tf.float32,
                                    initializer=tf.constant(np.load("DehazeNet/numpy_parameters/conv1_1.npy")))
        self.conv_1 = tf.nn.conv2d(self.batch_inputs, self.filters_1, [1, 1, 1, 1], padding="VALID")
        print(self.conv_1)
        self.output_of_relu_of_sum_of_conv_1_AND_bias_1 = tf.nn.relu(tf.nn.bias_add(self.conv_1, self.bias_1))

        def maxout(inputs, num_units, axis):
            inputs = ops.convert_to_tensor(inputs)
            shape = inputs.get_shape().as_list()
            num_channels = shape[axis]
            shape[axis] = -1
            shape += [num_channels // num_units]
            for i in range(len(shape)):
                if shape[i] is None:
                    shape[i] = gen_array_ops.shape(inputs)[i]
            outputs = math_ops.reduce_max(gen_array_ops.reshape(inputs, shape), -1, keep_dims=False)
            return outputs
        
        self.max_1 = maxout(tf.reshape(self.output_of_relu_of_sum_of_conv_1_AND_bias_1, [1, 224+11, 224+11, 20]), 4, 3)

        self.filters_2 = tf.get_variable("DehazeNet/filters_2",
                                    dtype=tf.float32,
                                    initializer=tf.constant(np.load("DehazeNet/numpy_parameters/conv2-1x1_0.npy"), shape=[1, 1, 4, 16]))
        
        self.bias_2 = tf.get_variable("DehazeNet/bias_2",
                                dtype=tf.float32,
                                initializer=tf.constant(np.load("DehazeNet/numpy_parameters/conv2-1x1_1.npy")))
        self.conv_2 = tf.nn.conv2d(tf.reshape(self.max_1, [1, 224+11, 224+11, 4]), self.filters_2, [1, 1, 1, 1], padding="VALID")
        self.output_of_relu_of_sum_of_conv_2_AND_bias_2 = tf.nn.relu(tf.nn.bias_add(self.conv_2, self.bias_2))
        
        self.filters_3 = tf.get_variable("DehazeNet/filters_3",
                                        dtype=tf.float32,
                                        initializer=tf.constant(np.load("DehazeNet/numpy_parameters/conv2-3x3_0.npy"), shape=[3, 3, 4, 16]))
        
        self.bias_3 = tf.get_variable("DehazeNet/bias_3",
                                    dtype=tf.float32,
                                    initializer=tf.constant(np.load("DehazeNet/numpy_parameters/conv2-3x3_1.npy")))
        self.conv_3 = tf.nn.conv2d(tf.reshape(self.max_1, [1, 224+11, 224+11, 4]), self.filters_3, [1, 1, 1, 1], padding="SAME")
        self.output_of_relu_of_sum_of_conv_3_AND_bias_3 = tf.nn.relu(tf.nn.bias_add(self.conv_3, self.bias_3))
        
        
        self.filters_4 = tf.get_variable("DehazeNet/filters_4",
                                        dtype=tf.float32,
                                        initializer=tf.constant(np.load("DehazeNet/numpy_parameters/conv2-5x5_0.npy"), shape=[5, 5, 4, 16]))
        
        self.bias_4 = tf.get_variable("DehazeNet/bias_4",
                                    dtype=tf.float32,
                                    initializer=tf.constant(np.load("DehazeNet/numpy_parameters/conv2-5x5_1.npy")))
        self.conv_4 = tf.nn.conv2d(tf.reshape(self.max_1, [1, 224+11, 224+11, 4]), self.filters_4, [1, 1, 1, 1], padding="SAME")
        self.output_of_relu_of_sum_of_conv_4_AND_bias_4 = tf.nn.relu(tf.nn.bias_add(self.conv_4, self.bias_4)) 


        self.filters_5 = tf.get_variable("DehazeNet/filters_5",
                                        dtype=tf.float32,
                                        initializer=tf.constant(np.load("DehazeNet/numpy_parameters/conv2-7x7_0.npy"), shape=[7, 7, 4, 16]))
        
        self.bias_5 = tf.get_variable("DehazeNet/bias_5",
                                    dtype=tf.float32,
                                    initializer=tf.constant(np.load("DehazeNet/numpy_parameters/conv2-7x7_1.npy")))
        self.conv_5 =  tf.nn.conv2d(tf.reshape(self.max_1, [1, 224+11, 224+11, 4]), self.filters_5, [1, 1, 1, 1], padding="SAME")
        self.output_of_relu_of_sum_of_conv_5_AND_bias_5 = tf.nn.relu(tf.nn.bias_add(self.conv_5, self.bias_5))
        print(self.output_of_relu_of_sum_of_conv_5_AND_bias_5)

        self.concat_convs_with_16_filters = tf.concat([self.output_of_relu_of_sum_of_conv_2_AND_bias_2,
                                                        self.output_of_relu_of_sum_of_conv_3_AND_bias_3,
                                                        self.output_of_relu_of_sum_of_conv_4_AND_bias_4,
                                                        self.output_of_relu_of_sum_of_conv_5_AND_bias_5], axis=3)
        self.max_pool_1 = tf.nn.max_pool(tf.nn.relu(self.concat_convs_with_16_filters), [1, 8, 8, 1], [1, 1, 1, 1], padding="VALID")
        self.filters_final = tf.get_variable("DehazeNet/filters_final",
                                            dtype=tf.float32,
                                            initializer=tf.constant(np.load("DehazeNet/numpy_parameters/ip1_0.npy"), shape=[5, 5, 64, 1]))
        self.bias_final = tf.get_variable("DehazeNet/bias_final",
                                             dtype=tf.float32,
                                             initializer=tf.constant(np.load("DehazeNet/numpy_parameters/ip1_1.npy")))
        self.conv_final = tf.nn.conv2d(self.max_pool_1, self.filters_final, [1, 1, 1, 1], padding="VALID")
        self.transmission = tf.reshape(tf.nn.relu(tf.nn.bias_add(self.conv_final, self.bias_final)), [1, 224, 224, 1])
        self.average_pool = tf.layers.average_pooling2d(self.transmission, (4, 4), (4, 4), padding='VALID')
        self.average_pool_flat = tf.reshape(self.average_pool, [-1, 56*56])

        self.dense = tf.layers.dense(inputs=self.average_pool_flat, units=1, activation=tf.nn.relu)

In [4]:
img = cv2.resize(cv2.cvtColor(cv2.imread("DehazeNet/img/canon.jpg"), cv2.COLOR_BGR2RGB), (224+15, 224+15))
img  = (img * 1.0) / (255 * 1.0)
img_tensor = tf.reshape(tf.convert_to_tensor(img, dtype=tf.float32), [1, 224+15, 224+15, 3])
model = DeHazeNetCNNModelTransmission(img_tensor)
transmission = None
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    sess.run(tf.tables_initializer())
    transmission = sess.run(model.transmission)
    transmission = transmission.reshape((224, 224))

Tensor("Conv2D:0", shape=(1, 235, 235, 20), dtype=float32)
Tensor("Relu_4:0", shape=(1, 235, 235, 16), dtype=float32)
Tensor("dense/Relu:0", shape=(1, 1), dtype=float32)


In [5]:
cv2.imshow("Transmission", transmission)
cv2.waitKey(0)

27

In [4]:
import re
all_variables = tf.contrib.framework.get_variables('DehazeNet')
variables_to_be_trained = tf.contrib.framework.get_variables('DehazeNet/final')
pre_trained_variables = [x for x in all_variables if re.match(r'DehazeNet/final.*', x.name) == None]
print(len(variables_to_be_trained))
print(len(pre_trained_variables))
print(all_variables)

print(len(tf.trainable_variables()))


2
10
[<tf.Variable 'DehazeNet/filters_1:0' shape=(5, 5, 3, 20) dtype=float32_ref>, <tf.Variable 'DehazeNet/bias_1:0' shape=(20,) dtype=float32_ref>, <tf.Variable 'DehazeNet/filters_2:0' shape=(1, 1, 4, 16) dtype=float32_ref>, <tf.Variable 'DehazeNet/bias_2:0' shape=(16,) dtype=float32_ref>, <tf.Variable 'DehazeNet/filters_3:0' shape=(3, 3, 4, 16) dtype=float32_ref>, <tf.Variable 'DehazeNet/bias_3:0' shape=(16,) dtype=float32_ref>, <tf.Variable 'DehazeNet/filters_4:0' shape=(5, 5, 4, 16) dtype=float32_ref>, <tf.Variable 'DehazeNet/bias_4:0' shape=(16,) dtype=float32_ref>, <tf.Variable 'DehazeNet/filters_5:0' shape=(7, 7, 4, 16) dtype=float32_ref>, <tf.Variable 'DehazeNet/bias_5:0' shape=(16,) dtype=float32_ref>, <tf.Variable 'DehazeNet/final_conv/kernel:0' shape=(5, 5, 64, 1) dtype=float32_ref>, <tf.Variable 'DehazeNet/final_conv/bias:0' shape=(1,) dtype=float32_ref>]
12


In [5]:
 tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
        loss = tf.losses.get_total_loss()

        # First we want to train only the reinitialized last layer fc8 for a few epochs.
        # We run minimize the loss only with respect to the fc8 variables (weight and bias).
        fc8_optimizer = tf.train.GradientDescentOptimizer(args.learning_rate1)
        fc8_train_op = fc8_optimizer.minimize(loss, var_list=fc8_variables)

        # Then we want to finetune the entire model for a few epochs.
        # We run minimize the loss only with respect to all the variables.
        full_optimizer = tf.train.GradientDescentOptimizer(args.learning_rate2)
        full_train_op = full_optimizer.minimize(loss)


[<tf.Variable 'DehazeNet/final_conv/kernel:0' shape=(5, 5, 64, 1) dtype=float32_ref>, <tf.Variable 'DehazeNet/final_conv/bias:0' shape=(1,) dtype=float32_ref>]


In [ ]:
#self.filter_1 = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, "conv2d/kernel")[0] #[5, 5, 3, 4]
#filter1_c = tf.split(self.filter_1, num_or_size_splits=4, axis=3)   # 4 x [5, 5, 3, 1]
#filter1_row0 = tf.concat(filter1_c[0:2], 0)   # [10, 5, 3, 1]
#filter1_row1 = tf.concat(filter1_c[2:4], 0)   # [10, 5, 3, 1]
#filter1_d = tf.concat([filter1_row0, filter1_row1], 1) # [10, 10, 3, 1]
#filter1_final_summary = tf.reshape(filter1_d, [1, 10, 10, 3])

#filter2 = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, "conv2d_1/kernel")[0] #[5, 5, 4, 8]
#filter2_pad= tf.zeros([5, 5, 4, 1], dtype=tf.float32)
#filter2_b = tf.concat([filter2, filter2_pad], 3)     #[5, 5, 4, 9]
#filter2_c = tf.split(filter2_b, num_or_size_splits=9, axis=3)                      # 9 x [5, 5, 4, 1]
#filter2_row0 = tf.concat(filter2_c[0:3], 0)    # [15, 5, 4, 1]
#filter2_row1 = tf.concat(filter2_c[5:10], 0)   # [15, 5, 4, 1]
#filter2_row2 = tf.concat(filter2_c[10:15], 0)  # [15, 5, 4, 1]
#filter2_d = tf.concat([filter2_row0, filter2_row1, filter2_row2], 1) # [15, 15, 4, 1]
#filter2_final_summary = tf.reshape(filter2_d, [1, 25, 25, 3])

In [7]:
import numpy
np.save("example_12313234234", 1)
a = np.load("example_12313234234.npy")
print(a)

1
